In [ ]:
!python -m spacy download en
!pip install transformers
!pip install torch
!pip install tensorflow
!python -m spacy download en_core_web_lg

In [59]:
import numpy as np
import re
import networkx as nx
import spacy
from summa import keywords
from summa.summarizer import summarize

# Laden des Spacy-Modells
import evaluate
import nltk
from nltk.tokenize import sent_tokenize
from transformers import AutoModelForSeq2SeqLM, PegasusForConditionalGeneration, PegasusTokenizer, AutoTokenizer
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import math
from transformers import PreTrainedTokenizerFast
from tqdm import tqdm
import random
from transformers import pipeline
import torch

In [60]:
df_train=pd.read_csv('data/data_train_with_features.csv')
df_train = df_train.drop('Unnamed: 0', axis=1)
df_train.head()

,classification,text,sentences,num_sentences,words,num_words,lemmas,stops,num_stops
0,Scientific,the interest in anchoring phenomena and phenom...,['the interest in anchoring phenomena and phen...,222,"['interest', 'anchoring', 'phenomena', 'phenom...",3533,"['interest', 'anchor', 'phenomenon', 'phenomen...","[the, in, and, in, has, been, by, their, in, a...",2426
1,Scientific,"galaxy clusters , as the largest peaks in the ...","['galaxy clusters , as the largest peaks in th...",372,"['galaxy', 'clusters', 'largest', 'peaks', 'co...",8982,"['galaxy', 'cluster', 'large', 'peak', 'cosmic...","[as, the, in, the, an, in, and, in, for, and, ...",5768
2,Scientific,quantum correlations between components of a s...,['quantum correlations between components of a...,138,"['quantum', 'correlations', 'components', 'sys...",2451,"['quantum', 'correlation', 'component', 'syste...","[between, of, a, or, are, on, the, of, whole, ...",1780
3,Scientific,methanol masers are often found in star - form...,['methanol masers are often found in star - fo...,141,"['methanol', 'masers', 'found', 'star', 'formi...",3232,"['methanol', 'maser', 'find', 'star', 'form', ...","[are, often, in, are, two, of, to, i, most, th...",1940
4,Scientific,interdisciplinary research has recently gained...,['interdisciplinary research has recently gain...,88,"['interdisciplinary', 'research', 'recently', ...",2695,"['interdisciplinary', 'research', 'recently', ...","[has, a, in, the, of, to, the, in, was, to, tw...",1997


In [61]:
df_train.classification.unique()

array(['Scientific', 'news', 'reviews', 'story'], dtype=object)

In [303]:

def reduce_repetitions(text):
    text = re.sub(r'\.{2,}', '.', text)
    text = re.sub(r'\!{2,}', '!', text)
    text = re.sub(r'\,{2,}', ',', text)
    text = re.sub(r'\;{2,}', ';', text)
    return text




def textrank_extractive(text, compression_rate=0.5,split='\. '):
    # Tokenisierung
    nlp = spacy.load("en_core_web_lg")
    #doc = nlp(text.replace("\n\n", " "))

    # Split the text at each ". " that is not followed by a single letter
    doc = re.split(fr'(?<!\b\w\w){split}', reduce_repetitions(re.sub(' +', ' ', text.replace("\n", " ").replace('-',' ').replace('_',' ').replace("\'", "").replace("!", ".").replace("?", ".").replace(";", ""))))
    sentences = [sent for sent in doc if len(sent.replace("-", " ").split()) > 2]

    # Speichern der Spacy-Dokumente der Sätze für spätere Verwendung
    sentence_docs = [nlp(sentence) for sentence in sentences]

    # Extrahiere Schlüsselsätze mit TextRank
    num_sentences = max(1, int(len(sentences) * compression_rate))
    extracted_sentences = summarize(text, words=num_sentences, split=True)

    # Erzeuge eine Matrix mit den Ähnlichkeiten zwischen den Sätzen
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    for i, doc_i in enumerate(sentence_docs):
        for j, doc_j in enumerate(sentence_docs):
            similarity = similarity_function(doc_i, doc_j)
            similarity_matrix[i, j] = similarity
    
    # Konstruiere einen Graphen mit den Sätzen als Knoten und den Ähnlichkeiten als Kanten
    graph = nx.from_numpy_array(similarity_matrix)

    # Berechne den TextRank-Score für jeden Satz
    scores = nx.pagerank_numpy(graph)

    # Wähle die besten Sätze basierend auf ihren TextRank-Scores aus
    top_sentences = sorted(scores, key=scores.get, reverse=True)[:num_sentences]

    # Sortiere die ausgewählten Sätze nach ihrer Position im Text
    top_sentences = sorted(top_sentences)

    # Gebe die extrahierten Schlüsselsätze zurück
    extracted_sentences = [sentences[index] for index in top_sentences]
    return extracted_sentences


def similarity_function(doc1, doc2):
    # Berechne die Cosinus-Ähnlichkeit zwischen den beiden Dokumenten
    similarity = doc1.similarity(doc2)
    return similarity




def compression_ratio(text, summary):
    # Berechne das Verhältnis der Anzahl der Wörter in der Zusammenfassung zur Anzahl der Wörter im Ausgangstext
    num_words_text = len(text.split())
    num_words_summary = len(summary.split())
    ratio = num_words_summary / num_words_text
    return ratio



def compression(text, compression_rate,split='\. '):
    max_iterations = 20
    iterations = 0
    #compression_rate -= 0.05
    
    extracted = textrank_extractive(text, compression_rate,split)
    summary = '. '.join(extracted)
    compression_rate_renwed = compression_rate


    while compression_ratio(text, summary) < compression_rate and iterations < max_iterations:
        iterations += 1
        compression_rate_renwed += 0.05
        if compression_rate_renwed > 1:
            compression_rate_renwed = 1
        extracted = textrank_extractive(text, compression_rate=compression_rate_renwed)
        summary = '. '.join(extracted)
    return summary




In [216]:
text_2=str(re.sub(' +', ' ', df_training[16:17]['text'][16].replace("\n", " ")))
textrank_extractive(text_2)

/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_89249/1902150806.py:28: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  scores = nx.pagerank_numpy(graph)
/Users/niclascramer/opt/miniconda3/envs/torch/lib/python3.9/site-packages/networkx/algorithms/link_analysis/pagerank_alg.py:354: FutureWarning: google_matrix will return an np.ndarray instead of a np.matrix in
NetworkX version 3.0.
  M = google_matrix(


['The young man was noble and brave, and every body hoped that he would some day be the King of England',
 'One summer Prince William went with his father across the sea to look after their lands in France',
 'They were wel comed with joy by all their people there, and the young prince was so gallant and kind, that he won the love of all who saw him',
 'The king, with his wise men and brave knights, set sail early in the day but Prince William with his younger friends waited a little while',
 'They had had so joyous a time in France that they were in no great haste to tear them selves away',
 'Then they went on board of the ship which was waiting to carry them home',
 'It was a beau ti ful ship with white sails and white masts, and it had been fitted up on purpose for this voyage',
 'The sea was smooth, the winds were fair, and no one thought of danger',
 'On the ship, every thing had been ar ranged to make the trip a pleasant one',
 'The sun had gone down before the white winged vesse

In [81]:
model_name='facebook/bart-large-cnn'
tokenizer = AutoTokenizer.from_pretrained(model_name)
summarizer = pipeline("text2text-generation", model=model_name)

In [336]:
def token_count(text):
    tokens = text.split()
    return len(tokens)


def adjust_length(text):
    length = token_count(text)
    if length <20:
        min_length = length + int(length * 0.05)
        max_length = min_length +min_length
    elif length <50:
        min_length = length + int(length * 0.05)
        max_length = min_length +min_length* 0.4
    elif length <60:
        min_length = length + int(length * 0.05)
        max_length = min_length +min_length* 0.25
    elif length < 80:
        min_length = length + int(length * 0.05)
        max_length = min_length + min_length* 0.25
    elif length < 100:
        min_length = length + int(length * 0.3)
        max_length = min_length + 100
    else:
        min_length = math.ceil(length / 50) * 70
        max_length = min_length + 100
    return min_length, max_length

In [65]:
split='\. '
sentenc='Whale naturalist and animal communicator Mary Getten goes beyond what other research has achieved in the way of whale study and communication processes: in 1991 she studied the J-Pod, a family of whales off Washington State. Her contention that they use direct telepathic communication between themselves and can do so with humans also makes for fascinating reading.Diane C. DonovanCalifornia Bookwatch'

sentences = re.split(fr'(?<!\b\w\w){split}', sentenc)
sentences

['Whale naturalist and animal communicator Mary Getten goes beyond what other research has achieved in the way of whale study and communication processes: in 1991 she studied the J-Pod, a family of whales off Washington State',
 'Her contention that they use direct telepathic communication between themselves and can do so with humans also makes for fascinating reading.Diane C',
 'DonovanCalifornia Bookwatch']

In [83]:
def batch_sent(sentenc,splitt=180,split='\. '):
    
    sentences = re.split(fr'(?<!\b\w\w){split}', sentenc.lower())

    # Erstellen Sie Batches von Sätzen, die weniger als 1024 Tokens enthalten
    batches = []
    batch = []
    batch_len = 0
    for sentence in sentences:
        
        sentence_len = len(tokenizer.tokenize(sentence))
        if sentence_len + batch_len > splitt:
            if sentence_len < splitt:  # überspringen Sie Sätze, die länger als 256 Tokens sind
                batches.append(batch)
                batch = [sentence]
                batch_len = sentence_len
            # wenn ein Satz alleine 1024 Tokens überschreitet, wird er übersprungen
        else:
            batch.append(sentence)
            batch_len += sentence_len
    batches.append(batch)
    return batches

In [319]:
text_news='this will be a waste of your time.'
ext_summary=summarizer(text_news, max_length=int(round(10*1,0)), min_length=int(round(8*1,0)),length_penalty=100,num_beams=2)
ext_summary

[{'generated_text': 'This will be a waste'}]

In [321]:
def paraphrase_of_text(df_s,text_name='text',komp_name='reduction_multiplier',split='\. '):

    df_summary_testing = pd.DataFrame(columns=['Zusammenfassung','Min_Kompressionsrate', 'Max_Kompressionsrate', 'Endgueltige_Kompressionsrate','länge Zusammenfassung','länge Ausgangstext','batch_texts','batch_output'])
    # Teilen Sie den Text in Sätze --> für tatsächliche umsetzubng
    soplitting=True
    
    for _, row in tqdm(df_s.iterrows(), total=df_s.shape[0]):
        batch_text_list=[]

        output_text_list=[]
        text = row[text_name]
        komp = row[komp_name]
        text_gesamt_list=[]

        for batch in tqdm(batch_sent(text,split=split), desc='Verarbeite Batches'):
            # Zusammenfügen der Sätze in einem Batch
            batch_text = '. '.join(batch)
            batch_text += "."

            min_length_test, max_length_test = adjust_length(batch_text)
            ext_summary=summarizer(batch_text, max_length=int(round(max_length_test*komp,0)), min_length=int(round(min_length_test*komp,0)),length_penalty=100,num_beams=2)
            # Erstellen Sie einen read_csv für die aktuellen Ergebnisse

            text_gesamt_list.append(ext_summary[0]['generated_text'])
         
        text_gesamt = '. '.join(text_gesamt_list)
        actual_compression_rate = len(text_gesamt.split(' '))/len(text.split(' '))*100
      
        df_current = pd.DataFrame({
            'Zusammenfassung': [text_gesamt],
            'Min_Kompressionsrate': [min_length_test],
            'Max_Kompressionsrate': [max_length_test],
            'Endgueltige_Kompressionsrate': [actual_compression_rate],
            'länge Zusammenfassung': [len(text_gesamt.split(' '))],
            'länge Ausgangstext': [len(text.split(' '))],
            'batch_texts': [batch_text_list],
            'batch_output': [text_gesamt_list]
        
        })
       
        # Fügen Sie die Daten zum DataFrame hinzu
        df_summary_testing = pd.concat([df_summary_testing, df_current], ignore_index=True)
      
    return df_summary_testing


In [297]:
def calculate_compression(df, total_tokens_col, current_tokens_col, desired_compression_rate):
    df['current_compression_rate'] = df[current_tokens_col] / df[total_tokens_col]
    df['compression_difference'] = df[desired_compression_rate] - df['current_compression_rate']
    df['reduction_multiplier'] = df[desired_compression_rate] / df['current_compression_rate']
    return df

In [217]:
def text_rank_algo(df,seed=10,split='\. '):
    df_return = pd.DataFrame(columns=['text','text_rank_text','tokens_gesamt', 'token_text_rank','desired_compression_rate','text_rank_compression_rate'])
    # Schleife zur Generierung der zufälligen Werte
    random.seed(seed)
    for index, row in tqdm(df.iterrows()):
        text = row["text"].replace("\n", " ")
        
        random_value = round(random.uniform(0.2, 0.8), 2)  # Zufälliger Wert zwischen 0.2 und 0.8 auf zwei Stellen nach dem Komma begrenzt
        text_rank_text= compression(text.replace("\n\n", " "),random_value,split)
        compression_ratio_value=compression_ratio(text, compression(text,random_value,split))
        text=re.sub(' +', ' ', text.replace("\n", " ").replace('-',' ').replace('_',' ').replace("\'", "").replace("!", ".").replace("?", ".").replace(";", ""))
        text_rank_text=re.sub(' +', ' ', text_rank_text.replace("\n", " ").replace('-',' ').replace('_',' ').replace("\'", "").replace("!", ".").replace("?", ".").replace(";", ""))
        df_current = pd.DataFrame({
                'text':[text],
                'text_rank_text': [text_rank_text],
                'tokens_gesamt': [len(text.split(' '))],
                'token_text_rank': [len(text_rank_text.split(' '))],
                'desired_compression_rate': [random_value],
                'text_rank_compression_rate': [compression_ratio_value]
            })
            # Fügen Sie die Daten zum DataFrame hinzu
        df_return = pd.concat([df_return, df_current], ignore_index=True)
    return df_return


In [86]:
def execute_text_gen(df,split='\. ',seed=10):
    rank_df=text_rank_algo(df,seed=seed,split=split)
    df_zwischen=calculate_compression(rank_df, 'tokens_gesamt', 'token_text_rank', 'desired_compression_rate')
    df_sum=paraphrase_of_text(df_zwischen[['text_rank_text','reduction_multiplier']],text_name='text_rank_text',split=split)
    merged_df = pd.concat([df_sum, df_zwischen], axis=1)
    merged_df['ent_com_rate']=merged_df['länge Zusammenfassung'] / merged_df['tokens_gesamt']
    return merged_df

In [71]:
df_train.classification.unique()


array(['Scientific', 'news', 'reviews', 'story'], dtype=object)

In [101]:
df_training=df_train[df_train.classification=='news'].reset_index(drop=True)[['classification','text']]

In [77]:
df_training[186:187]['text'][186]

'It was good horrifying never a dull moment u can watch and not be bored ir wondering when is a good part coming'

In [98]:
textrank_extractive(df_training[177:178]['text'][177],0.1)

['Whale naturalist and animal communicator Mary Getten goes beyond what other research has achieved in the way of whale study and communication processes: in 1991 she studied the J-Pod, a family of whales off Washington State']

In [95]:
re.split(fr'(?<!\b\w\w)\. ', df_training[177:178]['text'][177].replace("\n\n", " "))

['Whale naturalist and animal communicator Mary Getten goes beyond what other research has achieved in the way of whale study and communication processes: in 1991 she studied the J-Pod, a family of whales off Washington State',
 'Her contention that they use direct telepathic communication between themselves and can do so with humans also makes for fascinating reading.Diane C',
 'DonovanCalifornia Bookwatch']

In [117]:
test_df=df_train[df_train.classification=='news'].reset_index(drop=True)[['classification','text']][200:200+50]

In [163]:
for satz in test_df[36:37]['text'][236].split('.'):
    if len(satz)<2:
        print(satz)


In [ ]:
text.split()

In [ ]:

nlp = spacy.load("en_core_web_lg")
doc = nlp(df_training[67:68]['text'][67].replace("\n\n", " ").replace("-", " "))
[sent.text for sent in doc.sents if len(sent.text.replace("-", " ").split()) > 2]

In [106]:
df_training

,classification,text
0,story,"PART I\n\nIf you don't like Christmas stories,..."
1,story,The Railroad Journey was very long and slow. T...
2,story,His affair that night was prosy. He was intend...
3,story,"This is the tale which Jehan Tugluk Khan, a wi..."
4,story,THE fact that the man whom he feared had died ...
5,story,"IBRAHIM FADLALLAH shrugged bis shoulders: ""You..."
6,story,"BEHIND him the Koh Haji-Lal, the ""Mountains of..."
7,story,SPEAKING in after years about that period of h...
8,story,The ease and apparent willingness with which s...
9,story,Along in the 80's there occurred a question of...


In [188]:
re.sub(' +', ' ', df_training[16:17]['text'][16].replace("\n", " "))

'King Henry, the Handsome Scholar, had one son, named William, whom he dearly loved. The young man was noble and brave, and every-body hoped that he would some day be the King of England. One summer Prince William went with his father across the sea to look after their lands in France. They were wel-comed with joy by all their people there, and the young prince was so gallant and kind, that he won the love of all who saw him. But at last the time came for them to go back to England. The king, with his wise men and brave knights, set sail early in the day; but Prince William with his younger friends waited a little while. They had had so joyous a time in France that they were in no great haste to tear them-selves away. Then they went on board of the ship which was waiting to carry them home. It was a beau-ti-ful ship with white sails and white masts, and it had been fitted up on purpose for this voyage. The sea was smooth, the winds were fair, and no one thought of danger. On the ship, 

In [198]:
text_2='King Henry, the Handsome Scholar, had one son, named William, whom he dearly loved. The young man was noble and brave, and every-body hoped that he would some day be the King of England. One summer Prince William went with his father across the sea to look after their lands in France. They were wel-comed with joy by all their people there, and the young prince was so gallant and kind, that he won the love of all who saw him. But at last the time came for them to go back to England. The king, with his wise men and brave knights, set sail early in the day; but Prince William with his younger friends waited a little while. They had had so joyous a time in France that they were in no great haste to tear them-selves away. Then they went on board of the ship which was waiting to carry them home. It was a beau-ti-ful ship with white sails and white masts, and it had been fitted up on purpose for this voyage. The sea was smooth, the winds were fair, and no one thought of danger. On the ship, every-thing had been ar-ranged to make the trip a pleasant one. There was music and dancing, and everybody was merry and glad. The sun had gone down before the white-winged vessel was fairly out of the bay. But what of that? The moon was at its full, and it would give light enough; and before the dawn of the morrow, the narrow sea would be crossed. And so the prince, and the young people who were with him, gave themselves up to mer-ri-ment and feasting and joy. The ear-li-er hours of the night passed by; and then there was a cry of alarm on deck. A moment after-ward there was a great crash. The ship had struck upon a rock. The water rushed in. She was sinking. Ah, where now were those who had lately been so heart-free and glad? Every heart was full of fear. No one knew what to do. A small boat was quickly launched, and the prince with a few of his bravest friends leaped into it. They pushed off just as the ship was be-gin-ning to settle beneath the waves. Would they be saved? They had rowed hardly ten yards from the ship, when there was a cry from among those that were left behind. "Row back!" cried the prince."It is my little sister. She must be saved!" The men did not dare to disobey. The boat was again brought along-side of the sinking vessel. The prince stood up, and held out his arms for his sister. At that moment the ship gave a great lurch forward into the waves. One shriek of terror was heard, and then all was still save the sound of the moaning waters. Ship and boat, prince and prin-cess, and all the gay com-pa-ny that had set sail from France, went down to the bottom together. One man clung to a floating plank, and was saved the next day. He was the only person left alive to tell the sad story. When King Henry heard of the death of his son his grief was more than he could bear. His heart was broken. He had no more joy in life; and men say that no one ever saw him smile again. Here is a poem about him that your teacher may read to you, and perhaps, after a while, you may learn it by heart. HE NEVER SMILED AGAIN.There stood proud forms before his throne, The stately and the brave; But who could fill the place of one,-- That one beneath the wave? Before him passed the young and fair, In pleasure\'s reckless train; But seas dashed o\'er his son\'s bright hair-- He never smiled again. He sat where festal bowls went round; He heard the minstrel sing; He saw the tour-ney\'s victor crowned Amid the knightly ring. A murmur of the restless deep Was blent with every strain, A voice of winds that would not sleep-- He never smiled again. Hearts, in that time, closed o\'er the trace Of vows once fondly poured, And strangers took the kins-man\'s place At many a joyous board; Graves which true love had bathed with tears Were left to heaven\'s bright rain; Fresh hopes were born for other years-- _He_ never smiled again! '
type(text_2)

str

In [199]:
text_2=str(re.sub(' +', ' ', df_training[16:17]['text'][16].replace("\n", " ")))
type(text_2)

str

In [ ]:
text_2='King Henry, the Handsome Scholar, had one son, named William, whom he dearly loved. The young man was noble and brave, and every-body hoped that he would some day be the King of England. One summer Prince William went with his father across the sea to look after their lands in France. They were wel-comed with joy by all their people there, and the young prince was so gallant and kind, that he won the love of all who saw him. But at last the time came for them to go back to England. The king, with his wise men and brave knights, set sail early in the day; but Prince William with his younger friends waited a little while. They had had so joyous a time in France that they were in no great haste to tear them-selves away. Then they went on board of the ship which was waiting to carry them home. It was a beau-ti-ful ship with white sails and white masts, and it had been fitted up on purpose for this voyage. The sea was smooth, the winds were fair, and no one thought of danger. On the ship, every-thing had been ar-ranged to make the trip a pleasant one. There was music and dancing, and everybody was merry and glad. The sun had gone down before the white-winged vessel was fairly out of the bay. But what of that? The moon was at its full, and it would give light enough; and before the dawn of the morrow, the narrow sea would be crossed. And so the prince, and the young people who were with him, gave themselves up to mer-ri-ment and feasting and joy. The ear-li-er hours of the night passed by; and then there was a cry of alarm on deck. A moment after-ward there was a great crash. The ship had struck upon a rock. The water rushed in. She was sinking. Ah, where now were those who had lately been so heart-free and glad? Every heart was full of fear. No one knew what to do. A small boat was quickly launched, and the prince with a few of his bravest friends leaped into it. They pushed off just as the ship was be-gin-ning to settle beneath the waves. Would they be saved? They had rowed hardly ten yards from the ship, when there was a cry from among those that were left behind. "Row back!" cried the prince."It is my little sister. She must be saved!" The men did not dare to disobey. The boat was again brought along-side of the sinking vessel. The prince stood up, and held out his arms for his sister. At that moment the ship gave a great lurch forward into the waves. One shriek of terror was heard, and then all was still save the sound of the moaning waters. Ship and boat, prince and prin-cess, and all the gay com-pa-ny that had set sail from France, went down to the bottom together. One man clung to a floating plank, and was saved the next day. He was the only person left alive to tell the sad story. When King Henry heard of the death of his son his grief was more than he could bear. His heart was broken. He had no more joy in life; and men say that no one ever saw him smile again. Here is a poem about him that your teacher may read to you, and perhaps, after a while, you may learn it by heart. HE NEVER SMILED AGAIN.There stood proud forms before his throne, The stately and the brave; But who could fill the place of one,-- That one beneath the wave? Before him passed the young and fair, In pleasure\'s reckless train; But seas dashed o\'er his son\'s bright hair-- He never smiled again. He sat where festal bowls went round; He heard the minstrel sing; He saw the tour-ney\'s victor crowned Amid the knightly ring. A murmur of the restless deep Was blent with every strain, A voice of winds that would not sleep-- He never smiled again. Hearts, in that time, closed o\'er the trace Of vows once fondly poured, And strangers took the kins-man\'s place At many a joyous board; Graves which true love had bathed with tears Were left to heaven\'s bright rain; Fresh hopes were born for other years-- _He_ never smiled again! '
text_2=str(re.sub(' +', ' ', df_training[16:17]['text'][16].replace("\n", " ")))
textrank_extractive(text_2)

In [ ]:
textrank_extractive(re.sub(' +', ' ', df_training[16:17]['text'][16].replace("\n", " ").replace('-',' ').replace('_',' ').replace("\'", "").replace("!", ".").replace("?", ".").replace(";", "")))

In [175]:
len(re.sub(' +', ' ', df_training[16:17]['text'][16].replace("\n", " ").replace('-',' ').replace('_',' ').replace("\'", "").replace("!", ".").replace("?", ".").replace('"','').replace(";", "")).split())

758

In [ ]:
textrank_extractive(df_training[16:17]['text'][16].replace("\n", " ").replace('     ',' ').replace('    ',' ').replace('   ',' ').replace('--',' ').replace('_','').replace("\'", "").replace("!", ".").replace("?", ".").replace('"','').replace(";", ""))
#re.sub(' +', ' ', text)

In [ ]:
text=re.sub(' +', ' ', df_training[16:17]['text'][16].replace("\n", " ").replace('--',' ').replace('_',' ').replace("\'", ""))
textrank_extractive(text)


In [108]:
df_training=df_train[df_train.classification=='reviews'].reset_index(drop=True)[['classification','text']].copy()

In [109]:
df_training

,classification,text
0,reviews,This sound track was beautiful! It paints the ...
1,reviews,I'm reading a lot of reviews saying that this ...
2,reviews,This soundtrack is my favorite music of all ti...
3,reviews,I truly like this soundtrack and I enjoy video...
4,reviews,"If you've played the game, you know how divine..."
...,...,...
495,reviews,This book is a wonderful illustration of the s...
496,reviews,I think beyond the fact that Hazlitt produces ...
497,reviews,Economics in One Lesson elucidates the basic p...
498,reviews,"Economics in One Lesson is another great ""must..."


In [338]:
df_training=df_train[df_train.classification=='news'].reset_index(drop=True)[['classification','text']].copy()
alsdf_sum=execute_text_gen(df_training,split='\. ')
alsdf_sum.to_csv('data/ergbnisse/train_news.csv')

0it [00:00, ?it/s]/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_89249/1576956366.py:38: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  scores = nx.pagerank_numpy(graph)
/Users/niclascramer/opt/miniconda3/envs/torch/lib/python3.9/site-packages/networkx/algorithms/link_analysis/pagerank_alg.py:354: FutureWarning: google_matrix will return an np.ndarray instead of a np.matrix in
NetworkX version 3.0.
  M = google_matrix(
390it [36:03,  9.85s/it]

In [289]:
news_test=pd.read_csv('data/ergbnisse/train_news_test.csv')
#news_test[['batch_texts','batch_output']]
news_test['batch_output'][0]

"['The Administration of Union Territory Daman and Diu has revoked its order that made it compulsory for women to tie rakhis to their male colleagues on the occasion of Rakshabandhan on August 7.']"

In [290]:
news_test['text'][0]

'The Administration of Union Territory Daman and Diu has revoked its order that made it compulsory for women to tie rakhis to their male colleagues on the occasion of Rakshabandhan on August 7. The administration was forced to withdraw the decision within 24 hours of issuing the circular after it received flak from employees and was slammed on social media.'

In [291]:
news_test['batch_texts'][0]

"['The Administration of Union Territory Daman and Diu has revoked its order that made it compulsory for women to tie rakhis to their male colleagues on the occasion of Rakshabandhan on August 7']"

In [ ]:
df_training=df_train[df_train.classification=='news'].reset_index(drop=True)[['classification','text']].copy()
rank_df=text_rank_algo(df_training)
df_zwischen=calculate_compression(rank_df, 'tokens_gesamt', 'token_text_rank', 'desired_compression_rate')
alsdf_sum.to_csv(f'data/ergbnisse/train_news.csv')

In [220]:
seed=14
for x in range(0,401,100):
  df_training=df_train[df_train.classification=='reviews'].reset_index(drop=True)[['classification','text']][x:x+100].reset_index(drop=True).copy()
  rank_df=text_rank_algo(df_training,seed=seed)
  df_zwischen=calculate_compression(rank_df, 'tokens_gesamt', 'token_text_rank', 'desired_compression_rate')
  df_zwischen.to_csv(f'data/ergbnisse/train_story_text_rank_{x+100}.csv')
  seed+=1

0it [00:00, ?it/s]/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_89249/1902150806.py:28: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  scores = nx.pagerank_numpy(graph)
/Users/niclascramer/opt/miniconda3/envs/torch/lib/python3.9/site-packages/networkx/algorithms/link_analysis/pagerank_alg.py:354: FutureWarning: google_matrix will return an np.ndarray instead of a np.matrix in
NetworkX version 3.0.
  M = google_matrix(
100it [05:52,  3.52s/it]


In [311]:
df_train[df_train.classification=='reviews'].reset_index(drop=True)[416:417]['text'][416]

'my husband and I watched this and ended up watching something else. this will be a waste of your time'

In [ ]:
seed=10
for x in range(0,401,100):
  df_training=df_train[df_train.classification=='reviews'].reset_index(drop=True)[['classification','text']][x:x+100].reset_index(drop=True).copy()
  alsdf_sum=execute_text_gen(df_training,split='\. ',seed=seed)
  alsdf_sum.to_csv(f'data/ergbnisse/train_review_2_{x+100}.csv')
  seed+=1

0it [00:00, ?it/s]/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_89249/1576956366.py:38: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  scores = nx.pagerank_numpy(graph)
/Users/niclascramer/opt/miniconda3/envs/torch/lib/python3.9/site-packages/networkx/algorithms/link_analysis/pagerank_alg.py:354: FutureWarning: google_matrix will return an np.ndarray instead of a np.matrix in
NetworkX version 3.0.
  M = google_matrix(
100it [06:14,  3.74s/it]
 88%|████████▊ | 88/100 [06:58<00:57,  4.75s/it]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_89249/238664303.py:4 in <module>      │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_89249/238664303.py'                  │
│                                                                                                  │
│ /var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_89249/233606247.py:4 in               │
│ execute_text_gen                                                                                 │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_89249/233606247.py'                  │
│                                                                                                  │
│ /var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_89249/78837128.py:21 in               │
│ paraphrase_of_text                                                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_89249/78837128.py'                   │
│                                                                                                  │
│ /Users/niclascramer/opt/miniconda3/envs/torch/lib/python3.9/site-packages/transformers/pipelines │
│ /text2text_generation.py:150 in __call__                                                         │
│                                                                                                  │
│   147 │   │   │     ids of the generated text.                                                   │
│   148 │   │   """                                                                                │
│   149 │   │                                                                                      │
│ ❱ 150 │   │   result = super().__call__(*args, **kwargs)                                         │
│   151 │   │   if (                                                                               │
│   152 │   │   │   isinstance(args[0], list)                                                      │
│   153 │   │   │   and all(isinstance(el, str) for el in args[0])                                 │
│                                                                                                  │
│ /Users/niclascramer/opt/miniconda3/envs/torch/lib/python3.9/site-packages/transformers/pipelines │
│ /base.py:1074 in __call__                                                                        │
│                                                                                                  │
│   1071 │   │   elif is_iterable:                                                                 │
│   1072 │   │   │   return self.iterate(inputs, preprocess_params, forward_params, postprocess_p  │
│   1073 │   │   else:                                                                             │
│ ❱ 1074 │   │   │   return self.run_single(inputs, preprocess_params, forward_params, postproces  │
│   1075 │                                                                                         │
│   1076 │   def run_multi(self, inputs, preprocess_params, forward_params, postprocess_params):   │
│   1077 │   │   return [self.run_single(item, preprocess_params, forward_params, postprocess_par  │
│                                                            

In [95]:
df_sum[['Zusammenfassung', 'Min_Kompressionsrate', 'Max_Kompressionsrate',
       'länge Zusammenfassung',
       'text','tokens_gesamt',
       'token_text_rank', 'desired_compression_rate',
       'text_rank_compression_rate', 'current_compression_rate',
      'reduction_multiplier', 'ent_com_rate']]

,Zusammenfassung,Min_Kompressionsrate,Max_Kompressionsrate,länge Zusammenfassung,text,tokens_gesamt,token_text_rank,desired_compression_rate,text_rank_compression_rate,current_compression_rate,reduction_multiplier,ent_com_rate
0,The Administration of Union Territory Daman an...,34,84,33,The Administration of Union Territory Daman an...,60,33,0.54,0.550000,0.55,0.981818,0.55
1,Malaika Arora slammed an Instagram user who tr...,40,90,39,Malaika Arora slammed an Instagram user who tr...,60,39,0.46,0.650000,0.65,0.707692,0.65
2,Bihar Health Minister defined virgin as being ...,36,86,34,The Indira Gandhi Institute of Medical Science...,60,35,0.55,0.583333,0.583333,0.942857,0.566667
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,42,92,22,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,60,40,0.32,0.666667,0.666667,0.48,0.366667
4,Hotels in Maharashtra will train their staff t...,53,103,38,Hotels in Maharashtra will train their staff t...,60,51,0.69,0.850000,0.85,0.811765,0.633333
5,"Man, 32, was found hanging inside the washroom...",50,100,42,A 32-year-old man on Wednesday was found hangi...,60,48,0.69,0.800000,0.8,0.8625,0.7
6,Delhi High Court reduced the compensation awar...,37,87,35,The Delhi High Court reduced the compensation ...,61,36,0.59,0.600000,0.590164,0.999722,0.57377
7,60-year-old Dalit woman was allegedly lynched ...,27,77,21,A 60-year old Dalit woman was allegedly lynche...,60,26,0.30,0.433333,0.433333,0.692308,0.35
8,The Pawan Hans helicopter was flying at a crit...,63,113,40,An inquiry by the Aircraft Accident Investigat...,61,60,0.51,1.000000,0.983607,0.5185,0.655738
9,Congress party has opened a bank called 'State...,43,93,30,The Congress party has opened a bank called 'S...,57,41,0.40,0.719298,0.719298,0.556098,0.526316


In [115]:
df1=pd.DataFrame('data/ergbnisse/train_review_100.csv')
df2=pd.DataFrame('data/ergbnisse/train_review_200.csv')
df3=pd.DataFrame('data/ergbnisse/train_review_300.csv')
df4=pd.DataFrame('data/ergbnisse/train_review_400.csv')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_89249/3261862292.py:1 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_89249/3261862292.py'                 │
│                                                                                                  │
│ /Users/niclascramer/opt/miniconda3/envs/torch/lib/python3.9/site-packages/pandas/core/frame.py:7 │
│ 80 in __init__                                                                                   │
│                                                                                                  │
│     777 │   │   # For data is scalar                                                             │
│     778 │   │   else:                                                                            │
│     779 │   │   │   if index is None or columns is None:                                         │
│ ❱   780 │   │   │   │   raise ValueError("DataFrame constructor not properly called!")           │
│     781 │   │   │                                                                                │
│     782 │   │   │   index = ensure_index(index)                                                  │
│     783 │   │   │   columns = ensure_index(columns)                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: DataFrame constructor not properly called!

In [118]:

frames = [pd.read_csv('data/ergbnisse/train_review_100.csv'), pd.read_csv('data/ergbnisse/train_story_text_rank_200.csv'), pd.read_csv('data/ergbnisse/train_story_text_rank_300.csv'),pd.read_csv('data/ergbnisse/train_review_400.csv'),pd.read_csv('data/ergbnisse/train_review_500.csv')]

# Verkettung der DataFrames und Neuzuweisung der Indizes
df_review = pd.concat(frames, ignore_index=True).drop('Unnamed: 0', axis=1)

In [119]:
df_review

,Zusammenfassung,Min_Kompressionsrate,Max_Kompressionsrate,Endgueltige_Kompressionsrate,länge Zusammenfassung,länge Ausgangstext,text,text_rank_text,tokens_gesamt,token_text_rank,desired_compression_rate,text_rank_compression_rate,current_compression_rate,compression_difference,reduction_multiplier,ent_com_rate
0,This sound track was beautiful! It paints the ...,78,128,76.000000,57,75,This sound track was beautiful! It paints the ...,This sound track was beautiful! It paints the ...,75,75,0.54,1.000000,1.000000,-0.460000,0.540000,0.760000
1,The music is timeless and I'm been listening t...,58,108,100.000000,56,56,I'm reading a lot of reviews saying that this ...,The music is timeless and I'm been listening t...,91,56,0.46,0.615385,0.615385,-0.155385,0.747500,0.615385
2,"The soundtrack is amazing music, probably the ...",78,128,80.000000,60,75,This soundtrack is my favorite music of all ti...,"The higher energy tracks like ""Chrono Cross ~ ...",128,75,0.55,0.585938,0.585938,-0.035937,0.938667,0.468750
3,"On disk one Garden Of God, Chronopolis, Fates,...",64,114,40.983607,25,61,I truly like this soundtrack and I enjoy video...,I have played this game and most of the music ...,116,61,0.32,0.525862,0.525862,-0.205862,0.608525,0.215517
4,"The music is perfect if you ask me, the best i...",80,130,67.532468,52,77,"If you've played the game, you know how divine...","If you've played the game, you know how divine...",77,77,0.69,1.000000,1.000000,-0.310000,0.690000,0.675325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,Only thing i found funny about this movie was ...,35,85,97.058824,33,34,His movies seem to be going down hill. The onl...,The only thing i found funny about this movie ...,42,34,0.59,0.809524,0.809524,-0.219524,0.728824,0.785714
396,Comic has terminal illness that almost kills h...,42,92,70.000000,28,40,FUNNY PEOPLE is somewhat compelling and engagi...,FUNNY PEOPLE is somewhat compelling and engagi...,40,40,0.53,1.000000,1.000000,-0.470000,0.530000,0.700000
397,"""This movie seems like it's all over the place...",49,99,76.595745,36,47,"This is not a typical Adam Sandler movie, whic...","It's about ""comedy"" but I wouldn't consider th...",85,47,0.53,0.552941,0.552941,-0.022941,0.958511,0.423529
398,"I found the jokes and humor to be at best, lam...",48,98,102.173913,47,46,"Adam Sadler plays an unlikable comedian, Georg...","Everyone has sex but Ira, which seems to be a ...",174,46,0.22,0.264368,0.264368,-0.044368,0.832174,0.270115


In [221]:

frames = [pd.read_csv('data/ergbnisse/train_story_text_rank_100.csv'), pd.read_csv('data/ergbnisse/train_story_text_rank_200.csv'), pd.read_csv('data/ergbnisse/train_story_text_rank_300.csv'),pd.read_csv('data/ergbnisse/train_story_text_rank_400.csv'),pd.read_csv('data/ergbnisse/train_story_text_rank_500.csv')]

# Verkettung der DataFrames und Neuzuweisung der Indizes
df_story_text_rank = pd.concat(frames, ignore_index=True).drop('Unnamed: 0', axis=1)

In [222]:
df_story_text_rank.head()

,text,text_rank_text,tokens_gesamt,token_text_rank,desired_compression_rate,text_rank_compression_rate,current_compression_rate,compression_difference,reduction_multiplier
0,"PART I If you don't like Christmas stories, do...",For I warn you perfectly frankly that I am dis...,5499,3773,0.54,0.686125,0.686125,-0.146125,0.787029
1,The Railroad Journey was very long and slow. T...,And the Young Electrician who lolled across th...,5610,3999,0.46,0.712961,0.712834,-0.252834,0.645311
2,His affair that night was prosy. He was intend...,He was intending the murder of an old Spanish ...,5644,4526,0.55,0.801914,0.801914,-0.251914,0.685859
3,"This is the tale which Jehan Tugluk Khan, a wi...","This is the tale which Jehan Tugluk Khan, a wi...",4370,2414,0.32,0.552403,0.552403,-0.232403,0.579287
4,THE fact that the man whom he feared had died ...,THE fact that the man whom he feared had died ...,3412,3065,0.69,0.898300,0.898300,-0.208300,0.768117


In [223]:
df_sum=paraphrase_of_text(df_story_text_rank.head()[['text_rank_text','reduction_multiplier']],text_name='text_rank_text',split=split)


100%|██████████| 5/5 [25:30<00:00, 306.17s/it]


In [243]:
merged_df = pd.concat([df_sum, df_story_text_rank.head()], axis=1)
merged_df['ent_com_rate']=merged_df['länge Zusammenfassung'] / merged_df['tokens_gesamt']


/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_89249/1497145124.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['current_compression_rate'] = df[current_tokens_col] / df[total_tokens_col]
/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_89249/1497145124.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['compression_difference'] = df[desired_compression_rate] - df['current_compression_rate']
/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_89249/1497145124.py:4: Se

,Zusammenfassung,länge Zusammenfassung,text,tokens_gesamt,desired_compression_rate,ent_com_rate,current_compression_rate,compression_difference,reduction_multiplier
0,Flame's mother gasped when she found out who w...,3485,"PART I If you don't like Christmas stories, do...",5499,0.54,0.633752,0.633752,-0.093752,0.852069
1,The Young Electrician was neither one length n...,2680,The Railroad Journey was very long and slow. T...,5610,0.46,0.477718,0.477718,-0.017718,0.96291
2,He was intending the murder of an old Spanish ...,3803,His affair that night was prosy. He was intend...,5644,0.55,0.673813,0.673813,-0.123813,0.81625
3,"The tale of Vasantasena, the slave who was fre...",1295,"This is the tale which Jehan Tugluk Khan, a wi...",4370,0.32,0.296339,0.296339,0.023661,1.079846
4,Stuart McGregor reconstructed the final scene ...,2948,THE fact that the man whom he feared had died ...,3412,0.69,0.864009,0.864009,-0.174009,0.798602


In [ ]:
extr

In [244]:
merged_df

,Zusammenfassung,Min_Kompressionsrate,Max_Kompressionsrate,Endgueltige_Kompressionsrate,länge Zusammenfassung,länge Ausgangstext,text,text_rank_text,tokens_gesamt,token_text_rank,desired_compression_rate,text_rank_compression_rate,current_compression_rate,compression_difference,reduction_multiplier,ent_com_rate
0,Flame's mother gasped when she found out who w...,67,117,92.366817,3485,3773,"PART I If you don't like Christmas stories, do...",For I warn you perfectly frankly that I am dis...,5499,3773,0.54,0.686125,0.686125,-0.146125,0.787029,0.633752
1,The Young Electrician was neither one length n...,210,310,67.016754,2680,3999,The Railroad Journey was very long and slow. T...,And the Young Electrician who lolled across th...,5610,3999,0.46,0.712961,0.712834,-0.252834,0.645311,0.477718
2,He was intending the murder of an old Spanish ...,53,103,84.025630,3803,4526,His affair that night was prosy. He was intend...,He was intending the murder of an old Spanish ...,5644,4526,0.55,0.801914,0.801914,-0.251914,0.685859,0.673813
3,"The tale of Vasantasena, the slave who was fre...",60,110,53.645402,1295,2414,"This is the tale which Jehan Tugluk Khan, a wi...","This is the tale which Jehan Tugluk Khan, a wi...",4370,2414,0.32,0.552403,0.552403,-0.232403,0.579287,0.296339
4,Stuart McGregor reconstructed the final scene ...,67,117,96.182708,2948,3065,THE fact that the man whom he feared had died ...,THE fact that the man whom he feared had died ...,3412,3065,0.69,0.898300,0.898300,-0.208300,0.768117,0.864009


In [254]:
df_zwischen_test=calculate_compression(merged_df[['Zusammenfassung','länge Zusammenfassung','text','tokens_gesamt','desired_compression_rate','ent_com_rate']], 'tokens_gesamt', 'länge Zusammenfassung', 'desired_compression_rate')
df_zwischen_test

/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_89249/1497145124.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['current_compression_rate'] = df[current_tokens_col] / df[total_tokens_col]
/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_89249/1497145124.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['compression_difference'] = df[desired_compression_rate] - df['current_compression_rate']
/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_89249/1497145124.py:4: Se

,Zusammenfassung,länge Zusammenfassung,text,tokens_gesamt,desired_compression_rate,ent_com_rate,current_compression_rate,compression_difference,reduction_multiplier
0,Flame's mother gasped when she found out who w...,3485,"PART I If you don't like Christmas stories, do...",5499,0.54,0.633752,0.633752,-0.093752,0.852069
1,The Young Electrician was neither one length n...,2680,The Railroad Journey was very long and slow. T...,5610,0.46,0.477718,0.477718,-0.017718,0.96291
2,He was intending the murder of an old Spanish ...,3803,His affair that night was prosy. He was intend...,5644,0.55,0.673813,0.673813,-0.123813,0.81625
3,"The tale of Vasantasena, the slave who was fre...",1295,"This is the tale which Jehan Tugluk Khan, a wi...",4370,0.32,0.296339,0.296339,0.023661,1.079846
4,Stuart McGregor reconstructed the final scene ...,2948,THE fact that the man whom he feared had died ...,3412,0.69,0.864009,0.864009,-0.174009,0.798602


In [267]:
compression_ratio(df_zwischen_test['text'][4:5][4], compression(df_zwischen_test['Zusammenfassung'][4:5][4],df_zwischen_test['desired_compression_rate'][4:5][4],'\. '))

/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_89249/1902150806.py:28: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  scores = nx.pagerank_numpy(graph)


0.7069167643610785

In [ ]:
compression(df_zwischen_test['Zusammenfassung'][0:1][0],df_zwischen_test['reduction_multiplier'][0:1][0],'\. ')

In [248]:
len('. '.join(merged_df['Zusammenfassung'][4:5][4].split('. ')[:-40]).split())/merged_df['tokens_gesamt'][4:5][4]

0.6872801875732708

In [ ]:
rank_df=text_rank_algo(df,seed=seed,split=split)
df_zwischen=calculate_compression(rank_df, 'tokens_gesamt', 'token_text_rank', 'desired_compression_rate')